### Coletar os dados https://mainstream.ghan.nl/export.html // https://lastfm.ghan.nl/export/

# Bibliotecas necessárias

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import numpy as np
from datetime import datetime

# Pré-processamento

In [2]:
# Carregar a base de dados
df = pd.read_csv('/content/last.csv')

In [3]:
# Verificar colunas
df.columns

Index(['uts', 'utc_time', 'artist', 'artist_mbid', 'album', 'album_mbid',
       'track', 'track_mbid'],
      dtype='object')

In [4]:
# Separar em data e horário
df['data'] = pd.to_datetime(df['utc_time']).dt.date
df['horario'] = pd.to_datetime(df['utc_time']).dt.time

In [5]:
# Coletar ano, mês, dia e dia da semana
df['ano']  = pd.DatetimeIndex(df['data']).year
df['mes']  = pd.DatetimeIndex(df['data']).month_name()
df['dia']  = pd.DatetimeIndex(df['data']).day
df['dds']  = pd.DatetimeIndex(df['data']).day_name()

In [6]:
# Criar as colunas para hora, minutos e segundos
df[['hora','minutos','segundos']] = df['horario'].astype(str).str.split(':', expand=True).astype(int)

In [7]:
# Remover colunas desnecessárias
cols = ['artist_mbid','album_mbid','track_mbid','uts','utc_time','data','horario','minutos','segundos', 'dia']

for col in cols:
  df.drop(col, axis=1, inplace=True)

In [8]:
# Renomear colunas artist e track
df.rename(columns={'artist': 'artistas', 'track': 'faixas'}, inplace=True)

# Criar coluna id
df['id'] = range(0, len(df))

In [9]:
# Alterar os nomes dos meses para português
mes_mapping = {
    'January': 'Janeiro',
    'February': 'Fevereiro',
    'March': 'Março',
    'April': 'Abril',
    'May': 'Maio',
    'June': 'Junho',
    'July': 'Julho',
    'August': 'Agosto',
    'September': 'Setembro',
    'October': 'Outubro',
    'November': 'Novembro',
    'December': 'Dezembro',
}

df['mes'] = df['mes'].map(lambda x: mes_mapping[x])

In [10]:
# Alterar os nomes dos dias para português
dds_mapping = {
    'Sunday': 'Domingo',
    'Monday': 'Segunda',
    'Tuesday':'Terça', 
    'Wednesday':'Quarta', 
    'Thursday':'Quinta', 
    'Friday':'Sexta', 
    'Saturday':'Sábado',
}

df['dds'] = df['dds'].map(lambda x: dds_mapping[x])

In [11]:
# Remover dados com a data 1970, pois consiste em erro da plataforma
df.drop(df[df.ano == 1970].index, inplace=True)

## Criar as categorias

# Nova seção

In [12]:
# Artistas, álbuns e faixas 
contagemArtistas = df.artistas.value_counts().rename_axis('artista').reset_index(name='quantidade')
contagemAlbuns   = df.album.value_counts().rename_axis('album').reset_index(name='quantidade')
contagemFaixas   = df.groupby(['faixas', 'artistas']).agg({'id': 'count'}).sort_values(by='id', ascending= False).rename(columns={'id':'quantidade'}).reset_index()

In [13]:
# Ano, mês e dia da semana
contagemAnos = df.ano.value_counts().rename_axis('ano').reset_index(name='quantidade')
contagemMes  = df.mes.value_counts().rename_axis('mes').reset_index(name='quantidade')
contagemDDS  = df.dds.value_counts().rename_axis('dds').reset_index(name='quantidade')

# distribuição por horas
audicaoHoras  = df.hora.value_counts().rename_axis('hora').reset_index(name='quantidade').sort_values(by='hora')

In [14]:
# Evolução ao longo dos anos
acumuladoArtistas = df[df['artistas'].isin(contagemArtistas['artista'][:10])] #top 10 artistas
acumuladoArtistas = acumuladoArtistas.groupby(['artistas','ano']).count().groupby(level=[0]).cumsum()['faixas']
acumuladoArtistas = pd.DataFrame(acumuladoArtistas.unstack().T.fillna(method='ffill').T.stack()).rename(columns={0:'quantidade'}).reset_index()

In [15]:
# Principais artistas, álbuns e faixas em cada ano
artistasAno = df.groupby(['ano'])['artistas'].agg(pd.Series.mode).to_frame().reset_index()
albunsAno   = df.groupby(['ano'])['album'].agg(pd.Series.mode).to_frame().reset_index()
faixasAno   = df.groupby(['ano'])['faixas'].agg(pd.Series.mode).to_frame().reset_index()

# Funções

In [16]:
# Função para artistas, álbuns e faixas
def graficoContagens(df, x, y, title):
  fig = px.bar(data_frame=df, x= x, y=y, title=title, text_auto=True)

  fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    title_x=0.5,
    showlegend=False, 
    width=900, height=600, bargap=0.3, 
    yaxis=dict(
                title=None,
                tickfont_size=12,
                categoryorder= 'total ascending'),
    xaxis=dict(
               title='Scrobbles',
               titlefont_size=12,
               tickfont_size=12)
  )

  return fig.show()

# Gráfico de linhas para o ano
def graficoAno(df,x,y,title):

  df = df.sort_values(by=x)
  fig = px.line(df, x=x, y=y, title=title,  markers=True)

  fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    title_x=0.5,
    showlegend=False, 
    width=900, height=600,
    yaxis=dict(
                title=None,
                categoryorder ='category descending',
                tickfont_size=12,
                tickformat = ".0f"),
    xaxis=dict(
               title=None,
               dtick =1,
               titlefont_size=12,
               tickfont_size=12)
  )
    
  return fig.show()

# Scrobbles por mês
def graficoMes(df, x, y, title):

  meses = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

  fig = px.bar(data_frame=df, x= x, y=y, title=title, orientation='h', text_auto=".0f")

  fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    title_x=0.5,
    showlegend=False, 
    width=900, height=600, bargap=0.4, 
    yaxis=dict(
                title=None,
                tickfont_size=12),
    xaxis=dict(
               title='Scrobbles',
               titlefont_size=12,
               tickfont_size=12,
               tickformat = ".0f")
  )

  fig.update_yaxes(categoryarray= list(reversed(meses)))

  return fig.show()

# Scrobbles por dia da semana
def graficoDDS(df, x, y, title):
  dias = ['Domingo', 'Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado']

  fig = px.bar(data_frame=df, x= x, y=y, title=title, orientation='h', text_auto=".0f")

  fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    title_x=0.5,
    showlegend=False, 
    width=900, height=600, bargap=0.4, 
    yaxis=dict(
                title=None,
                tickfont_size=12,
               ),
    xaxis=dict(
               title='Scrobbles',
               titlefont_size=12,
               tickformat = ".0f",
               tickfont_size=12)
  )

  fig.update_yaxes(categoryarray= list(reversed(dias)))

  return fig.show()

# Gráfico com a evolução dos artistas ao longo dos anos
def cumulativoArtistas(data, x, y):
  data = [go.Scatter(dict(x=values[x], y=values[y], name=key,
                       mode='lines+markers')) for key, values in data]
  return iplot(data)

# Gráfico com os principais resultados por ano
def valoresAno(df, y, x):

  fig = px.scatter(data_frame = df, y = y, x = x)

  fig.update_layout(
    yaxis=dict(
               title=None,
               titlefont_size=12,
               tickfont_size=12),
    xaxis=dict(
               title=None,
               titlefont_size=12,
               dtick =1,
               tickfont_size=12),
    )

  fig.update_traces(marker_size=10)

  return fig.show()

# Gerar relógio para os scrobbles por horas
def graficoRelogio(df, x):
  #https://stackoverflow.com/questions/72595317/is-it-possible-to-generate-a-clock-chart-using-plotly
  r  = df[x].tolist()
  theta = np.arange(7.5,368,15)
  width = [15]*24

  ticktexts = [f'$\large{i}$' if i % 6 == 0 else '' for i in np.arange(24)]

  fig = go.Figure(go.Barpolar(
    r=r,
    theta=theta,
    width=width,
    marker_color=df[x],
    marker_colorscale='Blues',
    marker_line_color="white",
    marker_line_width=2,
    opacity=0.8
  ))

  fig.update_layout(
    template=None,
    polar=dict(
        hole=0.4,
        bgcolor='rgb(223, 223,223)',
        radialaxis=dict(
            showticklabels=False,
            ticks='',
            linewidth=2,
            linecolor='white',
            showgrid=False,
        ),
        angularaxis=dict(
            tickvals=np.arange(0,360,15),
            ticktext=ticktexts,
            showline=True,
            direction='clockwise',
            period=24,
            linecolor='white',
            gridcolor='white',
            showticklabels=True,
            ticks=''
        )
    )
  )

  return fig.show()

# Visualização

In [17]:
# Total de scrobbles por ano
graficoAno(contagemAnos, 'ano','quantidade', 'Scrobbles por ano')

In [18]:
# Artistas principais
graficoContagens(contagemArtistas[:15], 'quantidade', 'artista', 'Artistas principais')

In [19]:
# Álbuns principais
graficoContagens(contagemAlbuns[:15], 'quantidade', 'album', 'Álbuns principais')

In [20]:
# Faixas principais
graficoContagens(contagemFaixas[:15], 'quantidade', 'faixas', 'Faixas principais')

In [21]:
# Scrobbles por mês
graficoMes(contagemMes, 'quantidade','mes', 'Scrobbles por mês')

In [22]:
# Scrobbles por dia da semana
graficoDDS(contagemDDS, 'quantidade','dds', 'Scrobbles por dia da semana')

In [23]:
# Scrobble ao longo do dia
graficoRelogio(audicaoHoras, 'quantidade')

In [24]:
# Evolução dos artistas ao longo dos anos
artistaAgrupados = acumuladoArtistas.groupby('artistas') # agrupar os artistas
cumulativoArtistas(artistaAgrupados, 'ano', 'quantidade')

In [25]:
# Principais artistas por anos
valoresAno(artistasAno, "artistas", "ano")

In [26]:
# Principais álbuns por anos
valoresAno(albunsAno, "album", "ano")

In [27]:
# Principais faixas por anos
valoresAno(faixasAno, "faixas", "ano")